In [3]:
import torch
from diffusers import StableDiffusionPipeline
import os
from PIL import Image

In [5]:
os.makedirs("frames", exist_ok=True)

In [4]:
# --- Setup device ---
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# Load model (takes ~30s with GPU)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
# Your prompt
prompt = "A futuristic city at sunset with flying cars and neon lights, cinematic style"

In [8]:
# Generate multiple images (frames)
num_frames = 8
for i in range(num_frames):
    image = pipe(prompt).images[0]
    image.save(f"frames/frame_{i:03}.png")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
from moviepy.editor import ImageSequenceClip

# Create a video clip from the generated images
image_folder = "frames"
fps = 1  # 1 frame per second

clip = ImageSequenceClip([f"{image_folder}/frame_{i:03}.png" for i in range(num_frames)], fps=fps)
clip.write_videofile("video.mp4", codec='libx264')

  if event.key is 'enter':



Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4


In [12]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [13]:
from gtts import gTTS

# Use the same prompt as narration
tts = gTTS(text=prompt, lang='en')
tts.save("narration.mp3")

In [19]:
from moviepy.editor import AudioFileClip, ImageSequenceClip, concatenate_audioclips
from moviepy.audio.AudioClip import AudioClip

# Reload video
video_clip = ImageSequenceClip([f"{image_folder}/frame_{i:03}.png" for i in range(num_frames)], fps=fps)

# Load narration
narration = AudioFileClip("narration.mp3")

# Create silent audio clip for remaining duration
remaining_duration = max(0, video_clip.duration - narration.duration)

if remaining_duration > 0:
    silence = AudioClip(lambda t: 0, duration=remaining_duration).set_fps(narration.fps)
    audio_clip = concatenate_audioclips([narration, silence])
else:
    audio_clip = narration.subclip(0, video_clip.duration)

# Set audio and export
final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile("output_animation.mp4", codec='libx264', audio_codec='aac')




chunk:  80%|████████  | 142/177 [03:26<00:00, 797.60it/s, now=None]

chunk:  80%|████████  | 142/177 [03:26<00:00, 797.60it/s, now=None]

Moviepy - Building video output_animation.mp4.
MoviePy - Writing audio in output_animationTEMP_MPY_wvf_snd.mp4


                                                                   

chunk:  80%|████████  | 142/177 [03:26<00:00, 797.60it/s, now=None]

chunk:  80%|████████  | 142/177 [03:26<00:00, 797.60it/s, now=None]

MoviePy - Done.
Moviepy - Writing video output_animation.mp4





chunk:  80%|████████  | 142/177 [03:28<00:00, 797.60it/s, now=None]

chunk:  80%|████████  | 142/177 [03:28<00:00, 797.60it/s, now=None]

Moviepy - Done !
Moviepy - video ready output_animation.mp4


In [20]:
from google.colab import files
files.download(video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# --- Display video in notebook ---
from IPython.display import HTML
from base64 import b64encode
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

In [22]:
import torch
from diffusers import StableDiffusionPipeline
import os

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

os.makedirs("frames", exist_ok=True)

# Base prompt
base_prompt = "a girl dancing dressed like a cowgirl, western boots, hat, cinematic lighting, detailed"

num_frames = 16

# Generate images with different random seeds to simulate motion
for i in range(num_frames):
    generator = torch.Generator(device).manual_seed(42 + i*10)  # Change seed each frame
    image = pipe(base_prompt, generator=generator).images[0]
    image.save(f"frames/frame_{i:03}.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
from moviepy.editor import ImageSequenceClip

fps = 4  # low fps for demo
clip = ImageSequenceClip([f"frames/frame_{i:03}.png" for i in range(num_frames)], fps=fps)
clip.write_videofile("cowgirl_dance.mp4", codec="libx264")




chunk:  80%|████████  | 142/177 [08:26<00:00, 797.60it/s, now=None]

chunk:  80%|████████  | 142/177 [08:26<00:00, 797.60it/s, now=None]

Moviepy - Building video cowgirl_dance.mp4.
Moviepy - Writing video cowgirl_dance.mp4





chunk:  80%|████████  | 142/177 [08:26<00:00, 797.60it/s, now=None]

chunk:  80%|████████  | 142/177 [08:26<00:00, 797.60it/s, now=None]

Moviepy - Done !
Moviepy - video ready cowgirl_dance.mp4


In [25]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open("cowgirl_dance.mp4", 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")